# CommercialCompoundSearcher
An internal Sigman Lab tool for assessing the commercial availability of molecules based on the Pubchem database. We highly recommended to begin with a small subset (~25 molecules) to test the script first before using it on a larger dataset. Once a few variables are defined in later cells (like vendors to ignore), the script can be run autonomously by running all cells.


[https://github.com/thejameshoward/CommercialCompoundSearcher](https://github.com/thejameshoward/CommercialCompoundSearcher)

### Imports

In [ ]:
# Built ins
import urllib
from pathlib import Path
from pprint import pprint

# Data manipulation
import pandas as pd
import numpy as np

# Custom
from utils import canonicalize_smiles, smiles_to_inchi_key, smiles_to_inchi
from utils import remove_duplicate_inchi_keys
from utils import get_cid_from_inchi_key, get_vendor_list_from_cid
from utils import remove_specific_vendors_from_dataframe
from utils import draw_molecules_to_grid_image
from utils import convert_str_list
from utils import get_CAS_from_cid


## Reading in list of smiles

The list of smiles can be a plaintext document with __SMILES__ in the first line and the '.txt' extension. Alternatively, the file extension could be '.csv' and also contain a __SMILES__ column.

```
┌────────────────────────┐
│ SMILES                 │
│ CC(=O)OCC[N+](C)(C)C   │
│ CC(C[N+](C)(C)C)OC(=O) │
│ ...                    │
└────────────────────────┘

```


In [ ]:
# Define a file
file = Path('./data/small.txt')

# Read in the file
if file.suffix == '.txt':
    df = pd.read_table(file, header=0)
elif file.suffix == '.csv':
    df = pd.read_csv(file, header=0)
else:
    raise ValueError(f'{file.name} does not have a supported extension.')

# Check that the file is formatted correctly
assert 'SMILES' in df.columns

# Drop any empty rows
df.dropna(axis=0, how='any', inplace=True)

display(df)

## Canonicalization and additional molecular identifiers

This section is used to canonicalize the SMILES and add additional molecular identifier information using RDKit. The output of this block will contain warnings (and potentially errors) from RDKit. Many of these errors (such as None mol from RDKit) are handled by removing the SMILES string and storing it in a separate file. 

In [ ]:
# Apply canonicalization
#TODO Understand how this affects stereoisomerism in the SMILES/InChI/InChI key values
df['SMILES'] = df['SMILES'].apply(canonicalize_smiles)

# Add InChI column
df['INCHI'] = df['SMILES'].apply(smiles_to_inchi)

# Add InChI key column
df['INCHI_KEY'] = df['SMILES'].apply(smiles_to_inchi_key)

# Get every row that has np.nan values
failed = df[(df['INCHI'].isna()) | (df['INCHI_KEY'].isna())]

# Get every row that does not have np.nan values
df = df[~(df['INCHI'].isna()) | ~(df['INCHI_KEY'].isna())].copy(deep=True)

# Save the failed and the canonicalized datasets to a csv file
failed.to_csv('./results/failed_canonicalization.csv', index=False)
df.to_csv('./results/canonicalized.csv', index=False)

# Check if anything failed an notify the user
if failed.empty:
    print('No SMILES strings failed canonicalization.')
else:
    display(failed)

display(df)

## Remove duplicate InChI keys

Because we will be using REST queries to gather vendor information, it is important to remove duplicates because they will "waste" and REST query. This procedure removes __exact__ duplicates of the InChI key in the dataframe even if the SMILES string is different.

In [ ]:
# Remove exact duplicates
df, duplicates = remove_duplicate_inchi_keys(df=df)

# For your viewing pleasure
display(df)

if duplicates.empty:
    print('No duplicate entries were found.')
else:
    display(duplicates)

# Save the results for good book keeping.
df.to_csv('./results/added_molecular_identifiers.csv', index=False)
duplicates.to_csv('./results/duplicate_molecular_identifiers.csv', index=False)

## Query Pubchem for CID

The best identifier to use for querying Pubchem is the Pubchem Compound ID (CID). For more information on how Pubchem standardizes its database, please see the [compounds webpage](https://pubchem.ncbi.nlm.nih.gov/docs/compounds). This section will obtain a CID for a given InChi key. The REST queries each take at least 200 ms.

__If you stop this cell while it is running, you will lose all of your progress__

In [ ]:
# Get inchi keys as a list
inchi_keys = df['INCHI_KEY'].to_list()

# This assertion statement will fail if you have duplicate 
# InChi keys. If you don't care, remove the following line
assert len(list(set(inchi_keys))) == df.shape[0]

# Get the total length of InChI keys for tracking progress
total = len(inchi_keys)

# Enumerate over all inchi keys and add CID values
for i, inchi_key in enumerate(inchi_keys):

    print(f'Working on {i + 1} of {total} ({round((i + 1) / total * 100, 2)}%)')

    # Set cid to nan if we can't find it
    cid = np.nan

    # Try to get the CID, if there is no CID, skip
    try:
        cid = get_cid_from_inchi_key(inchi_key)
    except urllib.error.HTTPError as e:
        print(f'Could not convert InChi Key {inchi_key} to CID because {e}. Skipping.')
        continue

    # Check how many instances of that INCHI_KEY are in the df
    if df[df['INCHI_KEY'] == inchi_key].shape[0] != 1:
        print(f'WARNING: Found more than one InChI key {inchi_key}!')

    # Add the CID based on inchi_key
    df.loc[df['INCHI_KEY'] == inchi_key, 'CID'] = str(cid)

# Get the df of molecules for which there is no CID, save it for good book keeping
no_cids = df[df['CID'].astype(float).isna()]
no_cids.to_csv('./results/no_cid_found.csv', index=False)

# Get the new df that has CID values for each molecule
df = df[~(df['CID'].astype(float).isna())].copy(deep=True)

display(df)

## Query Pubchem for vendors

This section will us the CID values found in the previous cell to acquire a list of vendors from Pubchem. The REST queries each take at least 200 ms.

__If you stop this cell while it is running, you will lose all of your progress towards acquiring vendors__

In [ ]:
# Get inchi keys as a list
cids = df['CID'].astype(int).to_list()

# This assertion statement will fail if you have duplicate 
# InChi keys. If you don't care, remove the following line
assert len(list(set(cids))) == df.shape[0]

# Get the total number of CIDs for tracking progress
total = len(cids)

# Keep a list of CIDs that have no vendors
no_vendor_cids = []

# Make a CID vendor dictionary that will contain the
# PubchemVendor objects
cid_vendor_dict = {}

# Enumerate over all CIDs and look for vendors
for i, cid in enumerate(cids):
    print(f'Working on {i + 1} of {total} ({round((i + 1) / total * 100, 2)}%)')

    # Try to get the list of PubchemVendor objects
    try:
        vendors = list(set(get_vendor_list_from_cid(cid)))
    except urllib.error.HTTPError as e:
        print(f'Could not get vendor list from CID {cid}.')
        no_vendor_cids.append(int(cid))
        continue

    # Check how many instances of that CID are in the df
    if df[df['CID'].astype(int) == cid].shape[0] != 1:
        print(f'WARNING: Found more than one CID for {cid}!')

    # Add the CID/VENDORS based on inchi_key
    df.loc[df['CID'].astype(int) == cid, 'VENDORS'] = str([x.SourceName for x in vendors])

    # Add the CID:Vendor key:value pair
    cid_vendor_dict[cid] = vendors

# Get the df of molecules for which there are no vendors, save it for good book keeping
no_vendors = df[df['CID'].astype(int).isin(no_vendor_cids)].copy(deep=True)
no_vendors.to_csv('./results/no_vendors_found.csv')

# Get all the molecules that have vendors
df = df[~df['CID'].astype(int).isin(no_vendor_cids)]

display(df)

#print(cid_vendor_dict)

## Filtering Vendors

The term "commercial availability" may differ between applications. Some vendors report that a compound is purchasable but will only synthesize it upon request. Additionally, the geographic location of the vendor's warehouse may lead to extended shipping times. In this section, we can filter vendors by selecting them from a list of total vendors.

The next cells are organized into separate steps.

In [ ]:
# Print the total list of vendors
list_of_current_vendors = list(set([vendor for vendor_list in df['VENDORS'].apply(convert_str_list) for vendor in vendor_list]))
display(f'UNIQUE VENDORS:')
pprint(list_of_current_vendors)
print(f'\nN_UNIQUE_VENDORS: {len(list_of_current_vendors)}')


#### Select vendors to keep __OR__ vendors to remove

Two variables are declared below. Define one and only one of these variables to be a list of vendor strings. __This section relies on exact string comparison. Thus, it is important that the **exact** string is used from the block above.__ We recommend using VENDORS_TO_REMOVE to be more deliberate with vendor selection.

(experimental) We've included a list of vendors as a template for VENDORS_TO_KEEP.

In [ ]:
# Define only one of these as a list
VENDORS_TO_KEEP = ['TCI (Tokyo Chemical Industry)',
 'Ambeed',
 'Combi-Blocks',
 'Thermo Fisher Scientific',
 'Sigma-Aldrich',
 'VWR, Part of Avantor']

VENDORS_TO_REMOVE = None

# Convert the string representation of the list of vendors
# to an actual Python list
df['VENDORS'] = df['VENDORS'].apply(convert_str_list)

# Get the list of current vendors (again)
list_of_current_vendors = list(set([vendor for vendor_list in df['VENDORS'].apply(convert_str_list) for vendor in vendor_list]))

# Convert vendors to keep into a vendors_to_remove list
if VENDORS_TO_REMOVE is None and VENDORS_TO_KEEP is not None:
    VENDORS_TO_REMOVE = [x for x in list_of_current_vendors if x not in VENDORS_TO_KEEP]

# Illegal options
elif VENDORS_TO_REMOVE is not None and VENDORS_TO_KEEP is not None:
    raise ValueError(f'Define either VENDORS_TO_REMOVE or VENDORS_TO_KEEP as a list not both.')

# User not removing any vendors
elif VENDORS_TO_REMOVE is None and VENDORS_TO_KEEP is None:
    VENDORS_TO_REMOVE = []

else:
    raise ValueError(f'Make sure you define the unused variable at the beginning of this cell to None')

# Remove the unwanted vendors
df = remove_specific_vendors_from_dataframe(df, vendors=VENDORS_TO_REMOVE)

# Purge empty df entries now
#TODO Reevaluate use of string literals as list intermediates
df = df[~(df['VENDORS'].astype(str) == '[]')]

display(df)


'''
# TODO Get the links from the vendor information from pubchem
# Get the new list of vendors
list_of_current_vendors = list(set([vendor for vendor_list in df['VENDORS'].to_list() for vendor in vendor_list]))
print(f'UNIQUE VENDORS:')
pprint(list_of_current_vendors)
print(f'\nN_UNIQUE_VENDORS: {len(list_of_current_vendors)}')

display(df)

for col in df.columns:
    print(df[col].dtype)
'''

#### Save the curated list of molecules

In [ ]:
df.to_csv('./FINAL_LIBRARY_CURATED.csv', index=False)

## Query Pubchem for CAS number

This section will us the CID values found in the previous cells to acquire a CAS number from Pubchem. Often a molecule will have multiple CAS numbers and the `get_CAS_from_CID()` function will gather only one of them. Users should be aware that this looks for two dashes (-) in the numbers it receives from Pubchem to identify the CAS number. This procedure could be improved by a more systematic way of determining whether the item received from Pubchem is actually a CAS number. The REST queries each take at least 200 ms.

__If you stop this cell while it is running, you will lose all of your progress towards acquiring vendors__

In [ ]:
# Get the full list of CIDs from the library
cids = [int(x) for x in df['CID'].to_list() if x != '']

# Get the total number of CIDs for tracking progress
total = len(cids)

# Keep a list of CIDs that have no vendors
no_vendor_cids = []

# Enumerate over all CIDs and look for vendors
for i, cid in enumerate(cids):
    print(f'Working on {i + 1} of {total} ({round((i + 1) / total * 100, 2)}%)')

    # Try to get the list of PubchemVendor objects
    try:
        cas = get_CAS_from_cid(cid)
    except urllib.error.HTTPError as e:
        print(f'Could not get CAS number from CID {cid}. ERROR: {e}')
        continue

    # Add the CID/VENDORS based on inchi_key
    df.loc[df['CID'].astype(int) == cid, 'CAS_NUMBER'] = str(cas)

    # Check how many instances of that CID are in the df
    #if df[df['CID'].astype(int) == cid].shape[0] != 1:
    #    print(df[df['CID'] == cid])
    #    print(f'WARNING: Found more than one CID for {cid}!')
    ## Add the CID/VENDORS based on inchi_key
    #df.loc[df['CID'].astype(int) == cid, 'CAS'] = str(cas)

# Save the file 
df.to_csv('./FINAL_LIBRARY_CURATED_with_cas_numbers.csv', index=False)



## Query a Sigman Inventory Export for the CAS numbers

Export a full copy of the Sigman inventory in labsuit and point the inventory_spreadsheet variable to its path. This cell will create a slice of the dataframe that contains CAS numbers in both your curated library and the inventory spreadsheet.

In [ ]:
# Define the spreadsheet file
inventory_spreadsheet = Path('./data/Sigman-inventory-03-07-2024-example.xlsx')

# Read in the file (These settings should read the default format)
try:        
    inventory = pd.read_excel(file, header=0, sheet_name='Chemical', engine='openpyxl')
except Exception: # This is required because I think labsuit is not zipping their xlsx files
    with open(inventory_spreadsheet, 'rb') as infile:
        inventory = pd.read_excel(infile, sheet_name='Chemical')

# Filter inventory by presence of CAS
inventory['CAS_NUMBER'] = inventory['CAS_NUMBER'].astype(str).apply(str.strip)
inventory = inventory[inventory['CAS_NUMBER'].astype(str).isin(df['CAS_NUMBER'])]

display(inventory)

# Save the owned molecules in the results folder
inventory.to_csv('./results/owned_molecules.csv', index=False)

## Drawing molecules 🥳 !

In this section we've included some useful functions for drawing molecules in your library.

In [ ]:
# Get a list of all smiles
smiles = df['SMILES'].to_list()

print(f'Number of SMILES: {len(smiles)}')

# Get the PIL images of the grid by passing smiles list
images = draw_molecules_to_grid_image(smiles, mols_per_row=6, img_resolution=600)

for image in images:
    display(image)